<a href="https://colab.research.google.com/github/masterschool-content/retail_demand_forecast/blob/main/EDA_with_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all libraries we will need


In [ ]:
import pandas as pd

# Connect to Goole Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read the files with pandas

In [ ]:
# Load the CSV files into pandas DataFrames
df_stores = pd.read_csv('/content/drive/MyDrive/retail_kaggle_data/stores.csv')
df_items = pd.read_csv('/content/drive/MyDrive/retail_kaggle_data/items.csv')
df_transactions = pd.read_csv('/content/drive/MyDrive/retail_kaggle_data/transactions.csv')
df_oil = pd.read_csv('/content/drive/MyDrive/retail_kaggle_data/oil.csv')
df_holidays_events = pd.read_csv('/content/drive/MyDrive/retail_kaggle_data/holidays_events.csv')

#Note: train file is very large.
#To make sure we fit RAM avaible on Google-Colab, we will load only part of the file
#df_train = pd.read_csv('/content/drive/MyDrive/retail_kaggle_data/train.csv', nrows=1e7)

In [ ]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.6/242.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
#Better solution to work with large files: use Dask, a parallel computing library that lets you work with data larger than memory by splitting the computations into smaller tasks.
import dask.dataframe as dd

# Read the file with Dask (this doesn't load the data into memory immediately)
df_train = dd.read_csv('/content/drive/MyDrive/retail_kaggle_data/train.csv', blocksize='64MB', sample=100)
#df_train = dd.read_csv('/content/drive/MyDrive/retail_kaggle_data/train.csv')

# Perform computations with Dask
df_train.head(5)  # This triggers the computation to load the first few rows

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [ ]:
df_items.head(5)

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [ ]:
df_stores.head(5)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [ ]:
df_transactions.head(5)

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [ ]:
df_oil.head(5)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [ ]:
df_holidays_events.head(5)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


## Checking missing values

In [ ]:
# Count missing values per column
missing_values = df_train.isna().sum().compute()

# Print the result
print(missing_values)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


id                    0
date                  0
store_nbr             0
item_nbr              0
unit_sales            0
onpromotion    21657651
dtype: int64


In [ ]:
# Focusing on missing values in the 'onpromotion' column
df_train['onpromotion'] = df_train['onpromotion'].fillna(False).astype(bool)

### **Challenge 1: Checking for Missing Data**

Other files might contain the missing data too. Check each of them and and think what would be a best way to deal with such data (cleaning, filling up with default values or something else)?

# Handling Outliers

## Negative sales

In [ ]:
# Checking for negative sales (returns)
negative_sales = df_train[df_train['unit_sales'] < 0]

# Replacing negative sales with 0 to reflect returns as non-sales
df_train['unit_sales'] = df_train['unit_sales'].apply(lambda x: max(x, 0))

negative_sales.head()  # Viewing negative sales for analysis

/usr/local/lib/python3.10/dist-packages/dask_expr/_collection.py:4376: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('unit_sales', 'float64'))

  warnings.warn(meta_warning(meta))
/usr/local/lib/python3.10/dist-packages/dask/utils.py:1241: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return getattr(__obj, self.method)(*args, **kwargs)


,id,date,store_nbr,item_nbr,unit_sales,onpromotion
10655,10655,2013-01-02,10,456875,-3.0,False
46867,46867,2013-01-03,5,559044,-1.0,False
50970,50970,2013-01-03,9,365138,-3.0,False
71807,71807,2013-01-03,41,812716,-19.0,False
71992,71992,2013-01-03,41,1004551,-27.0,False


## Extremely high sales

In [ ]:
# Function to calculate Z-score for each group (store-item combination)
def calculate_store_item_zscore(group):
    # Compute mean and standard deviation for each store-item group
    mean_sales = group['unit_sales'].mean()
    std_sales = group['unit_sales'].std()

    # Calculate Z-score (avoiding division by zero for standard deviation)
    group['z_score'] = (group['unit_sales'] - mean_sales) / (std_sales if std_sales != 0 else 1)
    return group


# Apply the Z-score calculation to each store-item group without resetting the index
df_train_grouped = df_train.groupby(['store_nbr', 'item_nbr']).apply(calculate_store_item_zscore)
df_train_grouped = df_train_grouped.reset_index(drop=True)

# Define threshold for outliers (e.g., Z-score > 3)
outliers = df_train_grouped[df_train_grouped['z_score'] > 3].compute()

# Print summary
print(f"Number of outliers detected: {len(outliers)}")
outliers.head()

NameError: name 'df_train' is not defined

# Feature Engineering

In [ ]:
# Convert date column to datetime
df_train['date'] = dd.to_datetime(df_train['date'])
print(df_train['date'].min().compute(),df_train['date'].max().compute())

In [ ]:
# Creating new time-based features
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train['day'] = df_train['date'].dt.day
df_train['day_of_week'] = df_train['date'].dt.dayofweek

# Calculating rolling average of unit_sales
df_train['unit_sales_7d_avg'] = df_train.groupby(['item_nbr', 'store_nbr'])['unit_sales'].transform(lambda x: x.rolling(window=7).mean())

In [ ]:
#df_train.sample(frac=0.1).compute()

# Visualizing Time-Series Data

## Sales Over Time (Aggregated)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Aggregating total sales by date
sales_by_date = df_train.groupby('date')['unit_sales'].sum()

# Plotting the time-series
plt.figure(figsize=(12,6))
plt.plot(sales_by_date.index, sales_by_date.values)
plt.title('Total Unit Sales Over Time', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Unit Sales')
plt.show()

## Sales Trend by Year and Month

In [ ]:
# Aggregating sales by year and month
sales_by_month = df_train.groupby(['year', 'month'])['unit_sales'].sum().unstack()

# Plotting heatmap of sales by year and month
import seaborn as sns

plt.figure(figsize=(12,6))
sns.heatmap(sales_by_month, cmap='Blues', annot=True, fmt='.0f')
plt.title('Monthly Sales Trends Over Years', fontsize=16)
plt.show()

## Effect of Promotions on Sales

In [ ]:
# Aggregating promotion and non-promotion sales
promo_sales = df_train.groupby('onpromotion')['unit_sales'].sum()

# Plotting
plt.figure(figsize=(6,4))
promo_sales.plot(kind='bar', color=['blue', 'orange'])
plt.title('Sales With and Without Promotions', fontsize=16)
plt.ylabel('Unit Sales')
plt.show()

# Investigating Relationship with Oil Prices

In [ ]:
# Merging df_train with oil data on date
df_train_oil = pd.merge(df_train, df_oil, on='date', how='left')

# Plotting oil price vs unit sales
fig, ax1 = plt.subplots(figsize=(10,6))

ax1.set_xlabel('Date')
ax1.set_ylabel('Oil Price', color='tab:blue')
ax1.plot(df_train_oil['date'], df_train_oil['dcoilwtico'], color='tab:blue', label='Oil Price')
ax2 = ax1.twinx()
ax2.set_ylabel('Unit Sales', color='tab:green')
ax2.plot(df_train_oil['date'], df_train_oil['unit_sales'], color='tab:green', label='Unit Sales')

plt.title('Oil Price vs Unit Sales Over Time', fontsize=16)
plt.show()

# Examining the Impact of Holidays

In [ ]:
# Merging df_train data with holidays
df_train_holiday = pd.merge(df_train, df_holidays, on='date', how='left')

# Aggregating sales by holiday and non-holiday
holiday_sales = df_train_holiday.groupby('type')['unit_sales'].sum()

# Plotting holiday impact
plt.figure(figsize=(8,5))
holiday_sales.plot(kind='bar', color='green')
plt.title('Impact of Holidays on Sales', fontsize=16)
plt.ylabel('Total Unit Sales')
plt.show()

# Analyzing Perishable Items

In [ ]:
# Merging df_train with items to get perishable data
df_train_items = pd.merge(df_train, df_items, on='item_nbr', how='left')

# Aggregating sales by perishable and non-perishable items
perishable_sales = df_train_items.groupby('perishable')['unit_sales'].sum()

# Plotting sales for perishable and non-perishable items
plt.figure(figsize=(6,4))
perishable_sales.plot(kind='bar', color=['orange', 'green'])
plt.title('Sales of Perishable vs Non-Perishable Items', fontsize=16)
plt.ylabel('Total Unit Sales')
plt.show()